In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import math
import sympy
import scipy
from collections.abc import Callable

# 6. Clamped Cubic Spline

In [2]:
## ALGORITHM IN COURSE (FOR CLAMPED CUBIC SPLINE) ##
# zero vectors
def zeroV(m):
    z = [0]*m
    return(z)

#INPUT: n; x0, x1, ... ,xn; a0 = f(x0), a1 =f(x1), ... , an = f(xn), FPO=f'(x0), FPN=f'(xn).
def clamped_cubic_spline(x, y, FPO, FPN):
    """This function interpolates between the knots
       specified by lists x and y. The function computes the coefficients
       and outputs the solution of possible S(x)."""        
    
    n = len(x)
    a = y
    h = zeroV(n-1)

    # alpha will be values in a system of eq's that will allow us to solve for c
    # and then from there we can find b, d through substitution.
    alpha = zeroV(n)

    # l, u, z are used in the method for solving the linear system
    l = zeroV(n+1)
    u = zeroV(n)
    z = zeroV(n+1)

    # b, c, d will be the coefficients along with a.
    b = zeroV(n)     
    c = zeroV(n+1)
    d = zeroV(n)    

    for i in range(n-1):
        # h[i] is used to satisfy the condition that 
        # Si+1(xi+l) = Si(xi+l) for each i = 0,..,n-1
        # i.e., the values at the knots are "doubled up"
        h[i] = x[i+1]-x[i]  
        
    alpha[0] = 3*(a[1] - a[0])/h[0] - 3*FPO
    alpha[n-1] = 3*FPN - 3*(a[n-1] - a[n-2])/h[n-2]

    for i in range(1, n-1):
        # Sets up the linear system and allows us to find c.  Once we have 
        # c then b and d follow in terms of it.
        alpha[i] = (3./h[i])*(a[i+1]-a[i])-(3./h[i-1])*(a[i] - a[i-1])

    # I, II, (part of) III Sets up and solves tridiagonal linear system...
    # I   
    l[0] = 2*h[0]     
    u[0] = 0.5      
    z[0] = alpha[0]/l[0]

    # II
    for i in range(1, n-1):
        l[i] = 2*(x[i+1] - x[i-1]) - h[i-1]*u[i-1]
        u[i] = h[i]/l[i]
        z[i] = (alpha[i] - h[i-1]*z[i-1])/l[i]

    l[n] = h[n-2]*(2-u[n-2])
    z[n] = (alpha[n-1] - h[n-2]*z[n-2])/l[n]
    c[n] = z[n]

    # III... also find b, d in terms of c.
    for j in range(n-2, -1, -1):      
        c[j] = z[j] - u[j]*c[j+1]
        b[j] = (a[j+1] - a[j])/h[j] - h[j]*(c[j+1] + 2*c[j])/3.
        d[j] = (c[j+1] - c[j])/(3*h[j]) 
        
    for i in range(n-1): # Print out each equation if requested.
        factor = f"(x{')' if x[i] == 0 else f' - {x[i]:.2f})' }"
        print(f"s_{i}(x) = ", end='')
        if y[i] != 0:
            if y[i] < 0:
                print("-", end ='')
            print("%5.4f" % abs(y[i]), end = ' ')
        if b[i] != 0:
            print("-" if b[i] < 0 else "+", end =' ')
            print("%5.4f%s" % (abs(b[i]), factor), end = ' ')
        if c[i] != 0:
            print("-" if c[i] < 0 else "+", end =' ')
            print("%5.4f%s^2" % (abs(c[i]), factor), end = ' ')
        if d[i] != 0:
            print("-" if d[i] < 0 else "+", end =' ')
            print("%5.4f%s^3" % (abs(d[i]), factor))
        print("\n")
        
    return [y[:(n-1)], b[:(n-1)], c[:(n-1)], d[:(n-1)]]

In [3]:
## TEST
coeff_x = [0, 1, 2]
coeff_y = [0, 1, 2]
print(clamped_cubic_spline(x=coeff_x, y=coeff_y, FPO=1, FPN=1))

s_0(x) = + 1.0000(x) 

s_1(x) = 1.0000 + 1.0000(x - 1.00) 

[[0, 1], [1.0, 1.0], [0.0, 0.0], [0.0, 0.0]]


## (a).

Use a clamped cubic spline to predict the position of the car and its speed when $t = 10s$.

The equation of the clamped cubic splines are given by:
$$
s(x) = s_i(x) = a_i + b_i(x-x_i)+c_i(x-x_i)^2 + d_i(x-x_i)^3
$$
for $x$ in $[x_i, x_{i+1}]$.

In [4]:
coeff_x = [0, 3, 5, 8, 13]
coeff_y = [0, 225, 383, 623, 993]
print(clamped_cubic_spline(x=coeff_x, y=coeff_y, FPO=75, FPN=72))

s_0(x) = + 75.0000(x) - 0.6593(x)^2 + 0.2198(x)^3


s_1(x) = 225.0000 + 76.9779(x - 3.00) + 1.3186(x - 3.00)^2 - 0.1538(x - 3.00)^3


s_2(x) = 383.0000 + 80.4071(x - 5.00) + 0.3961(x - 5.00)^2 - 0.1773(x - 5.00)^3


s_3(x) = 623.0000 + 77.9975(x - 8.00) - 1.1993(x - 8.00)^2 + 0.0800(x - 8.00)^3


[[0, 225, 383, 623], [75.0, 76.97785977859779, 80.4071340713407, 77.99753997539976], [-0.6592865928659287, 1.3185731857318574, 0.3960639606396064, -1.1992619926199264], [0.21976219762197624, -0.15375153751537518, -0.1772584392510592, 0.0799507995079951]]


We obatain then:
$$
s_3(x) = 623.0000 + 77.9975(x - 8.00) - 1.1993(x - 8.00)^2 + 0.0800(x - 8.00)^3 \quad \text{for} \quad x \in [8,13]
$$
And 
$$
s^{\prime}_3(x) = 77.9975 - 2.3986(x - 8.00) + 0.24(x - 8.00)^2 \quad \text{for} \quad x \in [8,13]
$$
So that: $s_3(10)=774.8378$ and $s^{\prime}_3(10) = 74.16029999999999$.

Therefore, the clamped cubic spline predicts the position of the car and its speed when $t = 10s$ with $s_3(10)=774.84$ ft and the speed of $s^{\prime}_3(x) = 74.16$ ft/s.

In [5]:
S_3 = lambda x: 623.0000 + 77.9975*(x - 8.00) - 1.1993*(x - 8.00)**2 + 0.0800*(x - 8.00)**3
S_prime_3 = lambda x: 77.9975 - 2.3986*(x - 8.00) + 0.24*(x - 8.00)**2
print(S_3(10))
print(S_prime_3(10))

774.8378
74.16029999999999


## (b).

Use the derivative of the spline to determine whether the car ever exceeds a 55mi/h speed limit on the road; if so, what is the first time the car exceeds this speed?

First, let's take a look at the speed in mi/h. Note that the formula to convert from ft/s to mi/h is given by:

mi/h = ft/s $\times 0.681818$

\begin{array}{|c|c|c|c|c|c|}
\hline \text{Time} & 0 & 3  & 5 & 8 & 13 \\
\hline \text{Distance} & 0 & 225 & 383 & 623 & 993 \\
\hline \text{Speed (mi/h)}  & 51.13635 & 52.499986 & 54.54544 & 50.454532 & 49.090896 \\
\hline
\end{array}

In [6]:
def ft_s_TO_mi_h(x):
    return x*0.681818
list_speed_ft_s = [75, 77, 80, 74, 72]
list_speed_mi_h = [ft_s_TO_mi_h(s) for s in list_speed_ft_s]
print(list_speed_mi_h)

[51.13635, 52.499986, 54.54544, 50.454532, 49.090896]


To maximize the speed, we find the single critical point of $s^{\prime}(x)$, and compare the values of $s(x)$ at this point and the endpoint.

By using the table above, we will use the function $s_2(x)$ and we will find the critical point of the speed $s'_2(x)$ using the classical method, $s''_{2}(x)=0$. We have:
$$
s_2(x) = 383.0000 + 80.4071(x - 5.00) + 0.3961(x - 5.00)^2 - 0.1773(x - 5.00)^3
$$
So that:
$$
s'_2(x) = 80.4071+ 0.7922(x - 5.00) - 0.5319(x - 5.00)^2
$$
And
$$
s''_2(x) =0.7922 - 1.0638(x - 5.00)
$$
Since $s''_{2}(x)=0$, it implies that $x=5.745$.

We obtain then: $s'_2(5.745)=80.7$ ft/s $=55.024$ mi/h.

Therefore, we conclude that the speed $55$ mi/h was first exceeded at approximately $5.5 s$.

In [7]:
s_2 = lambda x: 383.0000 + 80.4071*(x - 5.00) + 0.3961*(x - 5.00)**2 - 0.1773*(x - 5.00)**3
s_prime_2 = lambda x: 80.4071+ 0.7922*(x - 5.00) - 0.5319*(x - 5.00)**2
print(s_2(5.745))
print(s_prime_2(5.745))
print(ft_s_TO_mi_h(s_prime_2(5.745)))

443.04982248278753
80.7020712025
55.02412478314614


## (c).

The predicted maximum speed for the car is:  $s'_2(5.745)=80.7$ ft/s $=55.024$ mi/h.